## Importando Bibliotecas

In [0]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() # criando uma SparkSession

## Leitura do Arquivo

In [0]:
arquivo = "/Volumes/curso_telos/analise_vendas/dados_brutos/online_retail_II.xlsx"

arq2011 = pd.read_excel(arquivo, sheet_name="Year 2010-2011")

df = arq2011.astype({'Invoice': str, 'StockCode': str, 'Description': str})

#Criando um dataframe do spark
df_vendas = spark.createDataFrame(df)

## Visualizando os tipos das colunas

In [0]:
df_vendas.printSchema()

root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Price: double (nullable = true)
 |-- Customer ID: double (nullable = true)
 |-- Country: string (nullable = true)



# Simulador 1 - Nível 8 (Big Data, ETL e Cloud para Data Analysts)

## 1. Quantos clientes distintos existem no dataset (com Customer ID não nulo)?

In [0]:
clientes_distintos = df_vendas.filter(F.col("Customer ID").isNotNull()).select("Customer ID").distinct().count()
display(clientes_distintos)

4372

## 2. Qual país (exceto Reino Unido) teve o maior número de clientes únicos?


In [0]:
#Utilizando o pySpark
pais_maior_clientes = df_vendas.filter((F.col("Customer ID").isNotNull()) & (F.col("Country") != "United Kingdom")).groupBy("Country").agg(F.countDistinct("Customer ID").alias("qtd_clientes")).orderBy(F.desc("qtd_clientes"))
pais_maior_clientes.show(10)


+-----------+------------+
|    Country|qtd_clientes|
+-----------+------------+
|    Germany|          95|
|     France|          87|
|      Spain|          31|
|    Belgium|          25|
|Switzerland|          21|
|   Portugal|          19|
|      Italy|          15|
|    Finland|          12|
|    Austria|          11|
|     Norway|          10|
+-----------+------------+
only showing top 10 rows


## 3. Qual comando PySpark pode ser usado para verificar o schema do DataFrame df criado a partir do dataset?

In [0]:
df_vendas.printSchema()

root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Price: double (nullable = true)
 |-- Customer ID: double (nullable = true)
 |-- Country: string (nullable = true)



## 4. Como você pode criar uma nova coluna TotalPrice no DataFrame df, que multiplica Quantity por UnitPrice?

In [0]:
df_vendas = df_vendas.withColumn("TotalPrice", F.col("Quantity") * F.col("Price"))
df_vendas.show(10)

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|        TotalPrice|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+------------------+
| 536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00| 2.55|    17850.0|United Kingdom|15.299999999999999|
| 536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00| 3.39|    17850.0|United Kingdom|             20.34|
| 536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00| 2.75|    17850.0|United Kingdom|              22.0|
| 536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00| 3.39|    17850.0|United Kingdom|             20.34|
| 536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00| 3.39|    17850.0|United Kingdom|             20.34|
| 536365|    22752|SET 7

In [0]:

df_vendas = df_vendas.withColumn("TotalPrice", F.col("TotalPrice").cast("double"))


In [0]:
df_vendas.printSchema()

root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Price: double (nullable = true)
 |-- Customer ID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- TotalPrice: double (nullable = true)



## 5. Qual país (exceto Reino Unido) teve o maior valor médio por transação (Total)?

In [0]:
media_transacao = df_vendas.filter((F.col("Country")!="United Kingdom")&F.col("TotalPrice").isNotNull()).groupBy("Country").agg(F.round(F.avg("TotalPrice"),2).alias("media_transacao")).orderBy(F.desc("media_transacao"))
media_transacao.show(10)

+-----------+---------------+
|    Country|media_transacao|
+-----------+---------------+
|Netherlands|         120.06|
|  Australia|         108.88|
|      Japan|          98.72|
|     Sweden|          79.21|
|    Denmark|          48.25|
|  Lithuania|          47.46|
|  Singapore|          39.83|
|    Lebanon|          37.64|
|     Brazil|          35.74|
|  Hong Kong|          35.13|
+-----------+---------------+
only showing top 10 rows


## 6. Qual o StockCode mais vendido (em quantidade) ao longo do período?

In [0]:
produto_mais_vendido = df_vendas.groupBy("StockCode").agg(F.sum("Quantity").alias("total_vendida")).orderBy(F.desc("total_vendida"))
produto_mais_vendido.show(10)

+---------+-------------+
|StockCode|total_vendida|
+---------+-------------+
|    22197|        56450|
|    84077|        53847|
|   85099B|        47363|
|   85123A|        38830|
|    84879|        36221|
|    21212|        36039|
|    23084|        30646|
|    22492|        26437|
|    22616|        26315|
|    21977|        24753|
+---------+-------------+
only showing top 10 rows


## 7. Qual cliente (ID) não nulo teve o maior volume total de compras (soma de Total)?

In [0]:
maior_compra_por_cliente = df_vendas.filter(F.col("Customer ID").isNotNull()).groupBy("Customer ID").agg(F.sum("Quantity").alias("Total_Compra")).orderBy(F.desc("Total_Compra")).limit(10)
maior_compra_por_cliente.show(10)

+-----------+------------+
|Customer ID|Total_Compra|
+-----------+------------+
|    14646.0|      196719|
|    12415.0|       77242|
|    14911.0|       77180|
|    17450.0|       69029|
|    18102.0|       64122|
|    17511.0|       63012|
|    13694.0|       61803|
|    14298.0|       58021|
|    14156.0|       57025|
|    16684.0|       49390|
+-----------+------------+



## 8. Qual mês do ano teve o maior faturamento total? Dica: converter a coluna InvoiceDate para mês.

In [0]:
faturamento_mes = df_vendas.groupBy(F.month("InvoiceDate").alias("mes")).agg(F.round(F.sum("TotalPrice")).alias("faturamento")).orderBy(F.desc("faturamento"))
faturamento_mes.show(10)

+---+-----------+
|mes|faturamento|
+---+-----------+
| 11|  1461756.0|
| 12|  1182643.0|
| 10|  1070705.0|
|  9|  1019688.0|
|  5|   723334.0|
|  6|   691123.0|
|  3|   683267.0|
|  8|   682681.0|
|  7|   681300.0|
|  1|   560000.0|
+---+-----------+
only showing top 10 rows


## 9. Qual descrição de produto aparece com maior frequência nas vendas?

In [0]:
descricao_produto = df_vendas.groupBy("Description").agg(F.count("*").alias("frequencia_descricao"), F.sum("Quantity").alias("total_quantidade_vendida")).orderBy(F.desc("frequencia_descricao")).limit(10)
descricao_produto.show(10)

+--------------------+--------------------+------------------------+
|         Description|frequencia_descricao|total_quantidade_vendida|
+--------------------+--------------------+------------------------+
|WHITE HANGING HEA...|                2369|                   35317|
|REGENCY CAKESTAND...|                2200|                   13033|
|JUMBO BAG RED RET...|                2159|                   47363|
|       PARTY BUNTING|                1727|                   18022|
|LUNCH BAG RED RET...|                1638|                   18779|
|ASSORTED COLOUR B...|                1501|                   36381|
|SET OF 3 CAKE TIN...|                1473|                    7336|
|                 nan|                1454|                  -13609|
|PACK OF 72 RETROS...|                1385|                   36039|
|LUNCH BAG  BLACK ...|                1350|                   12112|
+--------------------+--------------------+------------------------+

